In [5]:
import numpy as np;
import pandas as pd;
import scipy;
from scipy.linalg import inv;

In [6]:
df = pd.read_csv("prostate.data",header=0, delimiter="\t",);

In [7]:
df.head()

,Unnamed: 0,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


In [8]:
#drop first column
df.drop([df.columns[0]],inplace=True, axis='columns');
df.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 10 columns):
lcavol     97 non-null float64
lweight    97 non-null float64
age        97 non-null int64
lbph       97 non-null float64
svi        97 non-null int64
lcp        97 non-null float64
gleason    97 non-null int64
pgg45      97 non-null int64
lpsa       97 non-null float64
train      97 non-null object
dtypes: float64(5), int64(4), object(1)
memory usage: 7.7+ KB


In [10]:
df.describe()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,1.350010,3.628943,63.865979,0.100356,0.216495,-0.179366,6.752577,24.381443,2.478387
std,1.178625,0.428411,7.445117,1.450807,0.413995,1.398250,0.722134,28.204035,1.154329
min,-1.347074,2.374906,41.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,-0.430783
25%,0.512824,3.375880,60.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,1.731656
50%,1.446919,3.623007,65.000000,0.300105,0.000000,-0.798508,7.000000,15.000000,2.591516
75%,2.127041,3.876396,68.000000,1.558145,0.000000,1.178655,7.000000,40.000000,3.056357
max,3.821004,4.780383,79.000000,2.326302,1.000000,2.904165,9.000000,100.000000,5.582932


In [11]:
print(df.corr().round(3).to_latex())

\begin{tabular}{lrrrrrrrrr}
\toprule
{} &  lcavol &  lweight &    age &   lbph &    svi &    lcp &  gleason &  pgg45 &   lpsa \\
\midrule
lcavol  &   1.000 &    0.281 &  0.225 &  0.027 &  0.539 &  0.675 &    0.432 &  0.434 &  0.734 \\
lweight &   0.281 &    1.000 &  0.348 &  0.442 &  0.155 &  0.165 &    0.057 &  0.107 &  0.433 \\
age     &   0.225 &    0.348 &  1.000 &  0.350 &  0.118 &  0.128 &    0.269 &  0.276 &  0.170 \\
lbph    &   0.027 &    0.442 &  0.350 &  1.000 & -0.086 & -0.007 &    0.078 &  0.078 &  0.180 \\
svi     &   0.539 &    0.155 &  0.118 & -0.086 &  1.000 &  0.673 &    0.320 &  0.458 &  0.566 \\
lcp     &   0.675 &    0.165 &  0.128 & -0.007 &  0.673 &  1.000 &    0.515 &  0.632 &  0.549 \\
gleason &   0.432 &    0.057 &  0.269 &  0.078 &  0.320 &  0.515 &    1.000 &  0.752 &  0.369 \\
pgg45   &   0.434 &    0.107 &  0.276 &  0.078 &  0.458 &  0.632 &    0.752 &  1.000 &  0.422 \\
lpsa    &   0.734 &    0.433 &  0.170 &  0.180 &  0.566 &  0.549 &    0.369 &  0.422 &

# Data are highly correlated

In [12]:
y = df['lpsa'];
X = df.drop(['lpsa'],axis=1);
X.corr()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
lcavol,1.000000,0.280521,0.225000,0.027350,0.538845,0.675310,0.432417,0.433652
lweight,0.280521,1.000000,0.347969,0.442264,0.155385,0.164537,0.056882,0.107354
age,0.225000,0.347969,1.000000,0.350186,0.117658,0.127668,0.268892,0.276112
lbph,0.027350,0.442264,0.350186,1.000000,-0.085843,-0.006999,0.077820,0.078460
svi,0.538845,0.155385,0.117658,-0.085843,1.000000,0.673111,0.320412,0.457648
lcp,0.675310,0.164537,0.127668,-0.006999,0.673111,1.000000,0.514830,0.631528
gleason,0.432417,0.056882,0.268892,0.077820,0.320412,0.514830,1.000000,0.751905
pgg45,0.433652,0.107354,0.276112,0.078460,0.457648,0.631528,0.751905,1.000000


### Scaled data correlation

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing as pp
from sklearn.pipeline import Pipeline
std_scaler = StandardScaler();
X_train = X[X['train']=="T"];
y_train = y[X['train']=="T"];
X_test = X[X['train']=="F"];
y_test = y[X['train']=="F"];
X_train = X_train.drop('train',axis=1)
#X_scaled = pp.scale(X_train.astype('float')); # this works as the below one
X_scaled = std_scaler.fit_transform(X_train.astype('float'));
#put together scaled data and output to see the correlation
df_scaled = pd.concat([pd.DataFrame(X_scaled), y_train], axis=1, );
#-1 because df has still 'train' col
df_scaled.columns = df.columns[:-1];
df_scaled.corr()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
lcavol,1.000000,0.300232,0.286324,0.063168,0.592949,0.692043,0.426414,0.483161,0.727280
lweight,0.300232,1.000000,0.316723,0.437042,0.181054,0.156829,0.023558,0.074166,0.598021
age,0.286324,0.316723,1.000000,0.287346,0.128902,0.172951,0.365915,0.275806,0.345414
lbph,0.063168,0.437042,0.287346,1.000000,-0.139147,-0.088535,0.032992,-0.030404,0.396349
svi,0.592949,0.181054,0.128902,-0.139147,1.000000,0.671240,0.306875,0.481358,0.451672
lcp,0.692043,0.156829,0.172951,-0.088535,0.671240,1.000000,0.476437,0.662533,0.434572
gleason,0.426414,0.023558,0.365915,0.032992,0.306875,0.476437,1.000000,0.757056,0.331608
pgg45,0.483161,0.074166,0.275806,-0.030404,0.481358,0.662533,0.757056,1.000000,0.368055
lpsa,0.727280,0.598021,0.345414,0.396349,0.451672,0.434572,0.331608,0.368055,1.000000


In [14]:
print(df_scaled.corr().round(3).to_latex())

\begin{tabular}{lrrrrrrrrr}
\toprule
{} &  lcavol &  lweight &    age &   lbph &    svi &    lcp &  gleason &  pgg45 &   lpsa \\
\midrule
lcavol  &   1.000 &    0.300 &  0.286 &  0.063 &  0.593 &  0.692 &    0.426 &  0.483 &  0.727 \\
lweight &   0.300 &    1.000 &  0.317 &  0.437 &  0.181 &  0.157 &    0.024 &  0.074 &  0.598 \\
age     &   0.286 &    0.317 &  1.000 &  0.287 &  0.129 &  0.173 &    0.366 &  0.276 &  0.345 \\
lbph    &   0.063 &    0.437 &  0.287 &  1.000 & -0.139 & -0.089 &    0.033 & -0.030 &  0.396 \\
svi     &   0.593 &    0.181 &  0.129 & -0.139 &  1.000 &  0.671 &    0.307 &  0.481 &  0.452 \\
lcp     &   0.692 &    0.157 &  0.173 & -0.089 &  0.671 &  1.000 &    0.476 &  0.663 &  0.435 \\
gleason &   0.426 &    0.024 &  0.366 &  0.033 &  0.307 &  0.476 &    1.000 &  0.757 &  0.332 \\
pgg45   &   0.483 &    0.074 &  0.276 & -0.030 &  0.481 &  0.663 &    0.757 &  1.000 &  0.368 \\
lpsa    &   0.727 &    0.598 &  0.345 &  0.396 &  0.452 &  0.435 &    0.332 &  0.368 &

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing as pp
from sklearn.pipeline import Pipeline
std_scaler = StandardScaler();
X_train = X[X['train']=="T"];
y_train = y[X['train']=="T"];
X_test = X[X['train'] =="F"];
y_test = y[X['train'] =="F"];


In [16]:
#X_train = X;
#y_train = y;
X_train.drop(['train'], axis=1, inplace=True)
#del X, y
#### We assume data have been already shuffled.
#X_train.head()
X_scaled = pp.scale(X_train.astype('float'));
print(X_scaled.mean(axis=0))
print(X_scaled.std(axis=0))

[ 4.30832816e-17 -4.97114787e-18 -8.34324318e-16  6.62819716e-18
  3.97691830e-17 -4.30832816e-17  4.92143639e-16 -3.31409858e-17]
[1. 1. 1. 1. 1. 1. 1. 1.]


/Users/fra/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [17]:
print(X_train.describe());
print(pd.DataFrame(X_scaled).describe());
X_train.info()

          lcavol    lweight        age       lbph        svi        lcp  \
count  67.000000  67.000000  67.000000  67.000000  67.000000  67.000000   
mean    1.313492   3.626108  64.746269   0.071440   0.223881  -0.214203   
std     1.242590   0.476601   7.502208   1.463655   0.419989   1.400735   
min    -1.347074   2.374906  41.000000  -1.386294   0.000000  -1.386294   
25%     0.488279   3.330360  61.000000  -1.386294   0.000000  -1.386294   
50%     1.467874   3.598681  65.000000  -0.051293   0.000000  -0.798508   
75%     2.349065   3.883610  69.000000   1.547506   0.000000   0.994793   
max     3.821004   4.780383  79.000000   2.326302   1.000000   2.656757   

         gleason       pgg45  
count  67.000000   67.000000  
mean    6.731343   26.268657  
std     0.708864   29.301764  
min     6.000000    0.000000  
25%     6.000000    0.000000  
50%     7.000000   15.000000  
75%     7.000000   50.000000  
max     9.000000  100.000000  
                  0             1            

In [18]:
X_scaled = std_scaler.fit_transform(X_train.astype(float));
print(np.mean(X_scaled, axis=0));
print(np.std(X_scaled, axis=0));

[ 4.30832816e-17 -4.97114787e-18 -8.34324318e-16  6.62819716e-18
  3.97691830e-17 -4.30832816e-17  4.92143639e-16 -3.31409858e-17]
[1. 1. 1. 1. 1. 1. 1. 1.]


In [19]:
pd.DataFrame(X_scaled, columns=X_train.columns).corr()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
lcavol,1.000000,0.300232,0.286324,0.063168,0.592949,0.692043,0.426414,0.483161
lweight,0.300232,1.000000,0.316723,0.437042,0.181054,0.156829,0.023558,0.074166
age,0.286324,0.316723,1.000000,0.287346,0.128902,0.172951,0.365915,0.275806
lbph,0.063168,0.437042,0.287346,1.000000,-0.139147,-0.088535,0.032992,-0.030404
svi,0.592949,0.181054,0.128902,-0.139147,1.000000,0.671240,0.306875,0.481358
lcp,0.692043,0.156829,0.172951,-0.088535,0.671240,1.000000,0.476437,0.662533
gleason,0.426414,0.023558,0.365915,0.032992,0.306875,0.476437,1.000000,0.757056
pgg45,0.483161,0.074166,0.275806,-0.030404,0.481358,0.662533,0.757056,1.000000


In [20]:
X_train.columns

Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45'], dtype='object')

In [21]:
from sklearn.linear_model import LinearRegression
#X_b = np.c_[np.ones((len(X_scaled), 1)), X_scaled]


class CustomLinearRegression(LinearRegression):
    def __init__(self):
        super(CustomLinearRegression, self).__init__();
    def z_scores(self, y, X, bias=False):
        p = len(self.coef_);
        if (bias):
            y_pred = self.predict(X[:,1:]);
            X_with_bias = X;
        else:
            y_pred = self.predict(X);
            X_with_bias = np.c_[np.ones((X.shape[0], 1)), X];
        sigma_hat_sq = np.sum(np.square(y_train-y_pred))/(len(y_train)-p-1);
        std_err  = np.sqrt(np.diag(inv(X_with_bias.T.dot(X_with_bias)))*sigma_hat_sq);
        z_scores = np.multiply(np.append(self.intercept_, self.coef_), 1/std_err);
        return std_err, z_scores;
    def get_intercept_coef(self):
        return np.append(self.intercept_, self.coef_);
lr = CustomLinearRegression();
lr.fit( X_scaled, y_train);
lr.z_scores(y_train, X_scaled)


(array([0.08701959, 0.13250132, 0.10558798, 0.10135462, 0.1023518 ,
        0.12445059, 0.15364444, 0.14151003, 0.1583969 ]),
 array([28.18152744,  5.36629046,  2.75078939, -1.39590898,  2.05584563,
         2.46925518, -1.86691264, -0.14668121,  1.73783972]))

In [22]:
lr.coef_, lr.intercept_

(array([ 0.71104059,  0.29045029, -0.14148182,  0.21041951,  0.30730025,
        -0.28684075, -0.02075686,  0.27526843]), 2.4523450850746267)

In [23]:
from numpy.linalg import inv;
X_b = np.c_[np.ones((len(X_scaled), 1)), X_scaled];
print(np.mean(X_b, axis=0));
theta = inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_train);

[ 1.00000000e+00  4.30832816e-17 -4.97114787e-18 -8.34324318e-16
  6.62819716e-18  3.97691830e-17 -4.30832816e-17  4.92143639e-16
 -3.31409858e-17]


In [24]:
np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_train)

array([ 2.45234509,  0.71104059,  0.29045029, -0.14148182,  0.21041951,
        0.30730025, -0.28684075, -0.02075686,  0.27526843])

### Notice that the stochastic gradient coefficients are slightly different 

In [25]:
n_iter = 1000;
np.random.seed(42);
eta = 0.1
theta_gd = np.random.randn(X_b.shape[1],1);
for ii in range(n_iter):
    gradient =  -2/len(X_b)*X_b.T.dot(np.c_[y_train] - X_b.dot(theta_gd));
    theta_gd = theta_gd - eta*gradient;
    
print(pd.DataFrame(theta_gd.T, 
        columns=np.concatenate((np.array(["intercept"]), X_train.columns)))
      .round(3).to_latex(index=False));

\begin{tabular}{rrrrrrrrr}
\toprule
 intercept &  lcavol &  lweight &    age &  lbph &    svi &    lcp &  gleason &  pgg45 \\
\midrule
     2.452 &   0.711 &     0.29 & -0.141 &  0.21 &  0.307 & -0.287 &   -0.021 &  0.275 \\
\bottomrule
\end{tabular}



In [26]:
y_pred = lr.predict(X_scaled);
y_pred_man = X_b.dot(theta);

The **residual sum of squares (RSS)** and **Mean square error(MSE)**

In [27]:
from sklearn.metrics import mean_squared_error;
np.sum(np.square(y_train-y_pred))/len(y_pred), mean_squared_error(y_train, y_pred)

(0.43919976805833433, 0.43919976805833433)

In [28]:
from sklearn.metrics import mean_squared_error;
np.sum(np.square(y_train-y_pred_man))/len(y_pred), mean_squared_error(y_train, y_pred_man)

(0.43919976805833433, 0.43919976805833433)

In [29]:
# Apparently, scikit does not support statistical inference delibarately
def Z_score(y_train, theta, X_with_bias):
    p = X_with_bias.shape[1]-1;
    y_pred = X_with_bias.dot(theta);
    sigma_hat_sq = np.sum(np.square(y_train-y_pred))/(len(y_train)-p-1);
    std_err  = np.sqrt(np.diag(inv(X_with_bias.T.dot(X_with_bias)))*sigma_hat_sq);
    z_scores = np.multiply(theta, 1/std_err);
    return z_scores, std_err;

z_scores, std_err = Z_score(y_train, theta, X_b);

pd.DataFrame({"Coefficient": theta, "Std. error" : std_err,
              r"$Z$ score" : z_scores.ravel(),},
             index=pd.Index(["intercept"]).append(X_train.columns)
            ).round(2).head(10)

,Coefficient,Std. error,$Z$ score
intercept,2.45,0.09,28.18
lcavol,0.71,0.13,5.37
lweight,0.29,0.11,2.75
age,-0.14,0.10,-1.40
lbph,0.21,0.10,2.06
svi,0.31,0.12,2.47
lcp,-0.29,0.15,-1.87
gleason,-0.02,0.14,-0.15
pgg45,0.28,0.16,1.74


### Using the extended linear Regression class

In [30]:
std_err_lr, z_scores_lr = lr.z_scores(y_train, X_scaled);
pd.DataFrame({"Coefficient": lr.get_intercept_coef(), "Std. error" : std_err_lr,
              r"$Z$ score" : z_scores_lr},
             index=pd.Index(["intercept"]).append(X_train.columns)
            ).round(2).head(len(lr.coef_)+1)

,Coefficient,Std. error,$Z$ score
intercept,2.45,0.09,28.18
lcavol,0.71,0.13,5.37
lweight,0.29,0.11,2.75
age,-0.14,0.10,-1.40
lbph,0.21,0.10,2.06
svi,0.31,0.12,2.47
lcp,-0.29,0.15,-1.87
gleason,-0.02,0.14,-0.15
pgg45,0.28,0.16,1.74


In [31]:
print(pd.DataFrame({"Coefficient": lr.get_intercept_coef(), "Std. error" : std_err_lr,
              "$Z$ score" : z_scores_lr},
             index=pd.Index(["intercept"]).append(X_train.columns)
            ).round(2).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &  Coefficient &  Std. error &  \$Z\$ score \\
\midrule
intercept &         2.45 &        0.09 &      28.18 \\
lcavol    &         0.71 &        0.13 &       5.37 \\
lweight   &         0.29 &        0.11 &       2.75 \\
age       &        -0.14 &        0.10 &      -1.40 \\
lbph      &         0.21 &        0.10 &       2.06 \\
svi       &         0.31 &        0.12 &       2.47 \\
lcp       &        -0.29 &        0.15 &      -1.87 \\
gleason   &        -0.02 &        0.14 &      -0.15 \\
pgg45     &         0.28 &        0.16 &       1.74 \\
\bottomrule
\end{tabular}



# Best subset selection

In [32]:
#sklearn does not support
import itertools;

#intercept only
X_small = np.copy(X_b[:,0]).reshape(-1,1); #np.ones
b_man  = scipy.linalg.inv(X_small.T.dot(X_small)).dot(X_small.T.dot(y_train));
lr = CustomLinearRegression();
lr.fit(X_small, y_train);
y_pred_sk  = lr.predict(X_small);
y_pred_man = X_small.dot(b_man);
rss_sk  = np.sum(np.square(y_train-y_pred_sk));
rss_man = np.sum(np.square(y_train-y_pred_man.reshape(y_train.shape)));
num_features = X_b.shape[1]-1;
best_models = {num_features : dict()};
best_subset = dict();
best_subset['names']      = ["intercept",];
best_subset['indexes']    = [0]; #wrt X_with_bias_coeffs
best_subset['coeffs']     = b_man;
best_subset['err']        = rss_man;
errors = {"features":"intercept", "errors":rss_man};
best_subset['all_errors'] = errors;
best_subset['skl_model']  = lr;
best_models[0] = best_subset;
best_models["error_sequence"] = list();
best_models["error_sequence"].append(best_subset['err']);

for num_features in np.arange(1,len(X_train.columns)+1):
    err = np.infty;
    best_subset = dict();
    errors = {"features":list(), "errors":list()};
    
    for ff in list(itertools.combinations(X_train.columns, num_features)):
        #+1 because it will be used with the row of ones
        idx = [X_train.columns.get_loc(ii)+1 for ii in ff]; 
        idx.insert(0,0);
        X_small = X_b[:,idx];

        b_man  = inv(X_small.T.dot(X_small)).dot(X_small.T).dot(y_train);
        lr = LinearRegression();
        #print(X_small.shape)
        lr.fit(X_small[:,1:], y_train);
        y_pred_sk  = lr.predict(X_small[:,1:]);
        y_pred_man = X_small.dot(b_man);
        rss_sk  = np.sum(np.square(y_train-y_pred_sk));
        rss_man = np.sum(np.square(y_train-y_pred_man));
        
        errors["features"].append(ff);
        errors['errors'].append(rss_man);
        if rss_man < err:
            err = rss_man;
            best_subset['names']   = ff;
            best_subset['indexes'] = idx; #wrt X_with_bias_coeffs
            best_subset['coeffs']  = b_man;
            best_subset['err']     = rss_man/len(y_train);
            best_subset['all_errors'] = errors;
            best_subset['skl_model'] = lr;
            
      
    best_models["error_sequence"].append(best_subset['err']);
    best_models[num_features] = best_subset;


In [33]:
best_models[7]

{'names': ('lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'pgg45'),
 'indexes': [0, 1, 2, 3, 4, 5, 6, 8],
 'coeffs': array([ 2.45234509,  0.70781828,  0.29290477, -0.1450474 ,  0.20980704,
         0.30920631, -0.28557946,  0.26014081]),
 'err': 0.4393626913047325,
 'all_errors': {'features': [('lcavol',
    'lweight',
    'age',
    'lbph',
    'svi',
    'lcp',
    'gleason'),
   ('lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'pgg45'),
   ('lcavol', 'lweight', 'age', 'lbph', 'svi', 'gleason', 'pgg45'),
   ('lcavol', 'lweight', 'age', 'lbph', 'lcp', 'gleason', 'pgg45'),
   ('lcavol', 'lweight', 'age', 'svi', 'lcp', 'gleason', 'pgg45'),
   ('lcavol', 'lweight', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45'),
   ('lcavol', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45'),
   ('lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45')],
  'errors': [30.958629941880634,
   29.437300317417076,
   31.194688345903852,
   32.51981848057305,
   31.57070601748839,
   30.414990170034596,
   3

In [34]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import itertools;
from sklearn.pipeline import Pipeline;
from sklearn.preprocessing import StandardScaler;
from sklearn.base import clone;
from sklearn.metrics import mean_squared_error;

class error_sequence:
    def __init__(self, errors=tuple(), sequence=tuple()):
        self.errors = list(errors);
        self.names  = list(sequence);
    
    def printErrs(self):
        for err, el in zip(self.errors, self.names):
            print(str(el) + ": " + str(err));
    
    def addNewElement(self, names : tuple, val: float, idx : tuple):
        ids = tuple();
        if names:
            ids = tuple([names[el] for el in idx]);
        else:
            ids = idx;
        self.names.append(ids);
        self.errors.append(val,);

class best_submodel_abstract:
    def __init__(self):
        self._standardInit();
        #later let user choose type
        self.pipeline = Pipeline([
                ('Scaler', StandardScaler()), 
                ('LinearRegression',  CustomLinearRegression())]);  
    
    def get_scaler(self):
        return self.pipeline.named_steps['Scaler'];
    def get_model(self):
        return self.pipeline.named_steps['LinearRegression'];
        
        
    def _standardInit(self):
        self.num_features    = None;
        self.feature_names   = None;
        self.best_subset_idx = None; #wrt X_with_bias_coeffs
        self.best_error      = None;
        self.all_errors      = error_sequence();
        self.skl_model       = None;
        self.column_names    = None;
        
    def _removeBias(self, X, num_features, isItwithoutBias=True):
        self.num_features = num_features;
        if isinstance(X, pd.DataFrame):
            self.column_names = list(X.columns);
            X = X.values;
            
        if (not isItwithoutBias):
            # scikit will be used and LinearRegression does not want the column of 1s
            print("removing bias");
            X_small = X[:,1:];
        else:
            X_small = X;
        return X_small;
    
    def fit(self, X, y,num_features, isItwithoutBias=True):
        return;
    
    def _update_best_model(self, pipeline, subset, mse, col_names):
        self.pipeline        = pipeline;
        self.best_subset     = subset;
        self.best_subset_idx = subset; #wrt X_with_bias_coeffs
        self.best_error      = mse;
        self.model           = self.get_model();
        self.scaler          = self.get_scaler();
        if col_names:
            self.feature_names = col_names;
        return 
    
    def fit1Step(self, X_no_bias, y, idx:tuple)->(Pipeline, float):
        # copy to have the same type instead of declaring
        X_small = X_no_bias[:,idx]; 

        temp_pipe = clone(self.pipeline); 
        temp_pipe.fit(X_small, y); #probably separate scaler for transform
        y_pred_sk  = temp_pipe.predict(X_small);
        mse_sk     = mean_squared_error(y_train,y_pred_sk);
        return temp_pipe, mse_sk;

class best_submodel_sk(best_submodel_abstract):
    def __init__(self):
        super(best_submodel_sk, self).__init__();
        
    def fit(self, X, y, num_features, isItwithoutBias=True):
        X_no_bias   = self._removeBias(X,num_features, isItwithoutBias);
        err = np.infty;
        all_idx = [i for i in range(X_no_bias.shape[1])];
        errors = error_sequence();
        if num_features==0:
            temp_pipe, mse_sk = self.fit1Step(np.ones((X_no_bias.shape[0],1)), y, (0,));
            errors.addNewElement(("intercept",), mse_sk, (0,));
            self._update_best_model(temp_pipe, None, mse_sk, tuple(('intercept',)));
            return;
        elif num_features>X_no_bias.shape[1]:
            num_features=X_no_bias.shape[1];
        for ff in list(itertools.combinations(all_idx, num_features)):
            temp_pipe, mse_sk = self.fit1Step(X_no_bias, y, tuple(ff));
            errors.addNewElement(self.column_names, mse_sk, ff);
            if mse_sk < err:
                err = mse_sk;
                if self.column_names:
                    names = ["intercept"] + [self.column_names[ii] for ii in ff];
                self._update_best_model(temp_pipe, ff, mse_sk, tuple(names));
        self.all_errors = errors;

class AllModels:
    def __init__(self):
        self.bestModels = dict();
        
class AllBestModels(AllModels):
    def __init__(self):
        super(AllBestModels, self).__init__();
    def fit(self, X, y, isItwithoutBias=True):
        #+1: range ends before the last extreme: if without bias
        # one must add 1 to include all features.
        # if with bias the +1 is included because the matrix has the intercept
        max_num_feat = X.shape[1]+1*int(isItwithoutBias); 
        for el in range(0,max_num_feat):
            l = best_submodel_sk();
            l.fit(X, y, el);
            self.bestModels[el]=l;
            

In [35]:
a = AllBestModels()
a.fit(X_train, y_train);
a.bestModels[2].model.get_intercept_coef()

array([2.45234509, 0.77401717, 0.34927407])

# Forward stepwise selection

In [36]:
#sklearn does not support
import itertools;
from scipy.linalg import inv


print(X_b.shape)
#intercept only
num_features = X_b.shape[1]-1;
X_small = np.copy(X_b[:,0]).reshape(-1,1); #np.ones
b_man  = scipy.linalg.inv(X_small.T.dot(X_small)).dot(X_small.T.dot(y_train));
lr = LinearRegression();
lr.fit(X_small, y_train);
y_pred_sk  = lr.predict(X_small);
y_pred_man = X_small.dot(b_man);
rss_sk  = np.sum(np.square(y_train-y_pred_sk));
rss_man = np.sum(np.square(y_train-y_pred_man.reshape(y_train.shape)));
best_fw_models = {num_features : dict()};
best_fw_subset = dict();
best_fw_subset['names']      = ["intercept",];
best_fw_subset['indexes']    = [0]; #wrt X_with_bias_coeffs
best_fw_subset['coeffs']     = b_man;
best_fw_subset['err']        = rss_man;
best_fw_subset['all_errors'] = [];
best_fw_subset['skl_model']  = lr;
best_fw_models[0] = best_fw_subset.copy();
best_fw_models["error_sequence"] = list();
best_fw_models["error_sequence"].append(best_fw_subset['err']);
best_fw_subset['indexes'] = [0,];
l_feat = list(X_train.columns);
for num_features in np.arange(1,len(X_train.columns)+1):
    #print(best_fw_subset['coeffs'])
    err = np.infty;
    errors = {"features":list(), "errors":list()};
    best_fw_subset['names'].append('')
    best_fw_subset['indexes'].append(np.infty);
    for ff in l_feat:
        idx = best_fw_subset['indexes'][:-1];
        idx.append(X_train.columns.get_loc(ff)+1);
        X_small = X_b[:,idx];
        b_man = inv(X_small.T.dot(X_small)).dot(X_small.T).dot(y_train);
        lr = LinearRegression();
        lr.fit(X_small[:,1:], y_train);
        y_pred_sk  = lr.predict(X_small[:,1:]);
        y_pred_man = X_small.dot(b_man);

        rss_sk  = np.sum(np.square(y_train-y_pred_sk));
        rss_man = np.sum(np.square(y_train-y_pred_man));
        errors["features"].append(ff);
        errors['errors'].append(rss_man);
        if rss_man < err:
            err = rss_man;
            best_fw_subset['names'][-1]   = ff;
            best_fw_subset['indexes'][-1] = idx[-1]; #wrt X_with_bias_coeffs
            best_fw_subset['coeffs']      = b_man.copy();
            best_fw_subset['err']         = rss_man;
            best_fw_subset['all_errors']  = errors;
            best_fw_subset['skl_model']   = lr;

    l_feat.remove(best_fw_subset['names'][-1]);
    best_fw_models["error_sequence"].append(best_fw_subset['err']);
    best_fw_models[num_features] = best_fw_subset.copy();
    #print(num_features)

(67, 9)


In [37]:
best_fw_models[3]["coeffs"]

array([2.45234509, 0.64128976, 0.34852683, 0.22422118])

# Orthogonal dependent stepwise


In [38]:
#sklearn does not support
from scipy.linalg import inv

num_features = X_b.shape[1]-1;
print(X_b.shape)
#intercept only
X_small = np.copy(X_b[:,0]).reshape(-1,1); #np.ones
b_man  = scipy.linalg.inv(X_small.T.dot(X_small)).dot(X_small.T.dot(y_train));
lr = LinearRegression();
lr.fit(X_small, y_train);
y_pred_sk  = lr.predict(X_small);
print(X_small.shape)
y_pred_man = X_small.dot(b_man);
rss_sk  = np.sum(np.square(y_train-y_pred_sk));
rss_man = np.sum(np.square(y_train-y_pred_man.reshape(y_train.shape)));
best_fw_orth_dep_models = {num_features : dict()};
best_fw_orth_dep_subset = dict();
best_fw_orth_dep_subset['names']      = ["intercept",];
best_fw_orth_dep_subset['indexes']    = [0]; #wrt X_with_bias_coeffs
best_fw_orth_dep_subset['coeffs']     = b_man;
best_fw_orth_dep_subset['err']        = rss_man;
best_fw_orth_dep_subset['all_errors'] = [];
best_fw_orth_dep_subset['skl_model']  = lr;
best_fw_orth_dep_models[0] = best_fw_orth_dep_subset.copy();
best_fw_orth_dep_models["error_sequence"] = list();
best_fw_orth_dep_models["error_sequence"].append(best_fw_orth_dep_subset['err']);
best_fw_orth_dep_subset['indexes'] = [0,];
l_feat = list(X_train.columns);
y_new  = y_train;
for num_features in np.arange(1,len(X_train.columns)+1):
    err = np.infty;
    errors = {"features":list(), "errors":list()};
    best_fw_orth_dep_subset['names'].append('')
    best_fw_orth_dep_subset['indexes'].append(np.infty);
    for ff in l_feat:
        idx = best_fw_orth_dep_subset['indexes'][:-1];
        idx.append(X_train.columns.get_loc(ff)+1);
        X_small = X_b[:,idx];
        b_man = inv(X_small.T.dot(X_small)).dot(X_small.T).dot(y_train);
        lr = LinearRegression();
        lr.fit(X_small[:,1:], y_train);
        y_pred_sk  = lr.predict(X_small[:,1:]);
        y_pred_man = X_small.dot(b_man);
        rss_sk  = np.sum(np.square(y_train-y_pred_sk));
        rss_man = np.sum(np.square(y_train-y_pred_man));
        errors["features"].append(ff);
        errors['errors'].append(rss_man);
        if rss_man < err:
            err = rss_man;
            #print(best_fw_subset['names'])
            best_fw_orth_dep_subset['names'][-1]   = ff;
            best_fw_orth_dep_subset['indexes'][-1] = idx[-1]; #wrt X_with_bias_coeffs
            best_fw_orth_dep_subset['coeffs']      = b_man;
            best_fw_orth_dep_subset['err']         = rss_man;
            best_fw_orth_dep_subset['all_errors']  = errors;
            best_fw_orth_dep_subset['skl_model']   = lr;

    #here is the difference from normal forward stepwise
    y_new = y_new - y_pred;
    l_feat.remove(best_fw_orth_dep_subset['names'][-1]);
    best_fw_orth_dep_models["error_sequence"].append(best_fw_orth_dep_subset['err']);
    best_fw_orth_dep_models[num_features] = best_fw_orth_dep_subset.copy();

(67, 9)
(67, 1)


In [39]:
best_models[3]["coeffs"]

array([2.45234509, 0.64128976, 0.34852683, 0.22422118])

# QR forward stepwise

In [40]:
#sklearn does not support
from scipy.linalg import inv
import scipy.linalg
import scipy.linalg

print(X_b.shape)

num_features = X_b.shape[1]-1;
l_feat = list(X_train.columns);
y_new  = y_train;

q1  = X_b[:,0];
q1  = (q1/scipy.linalg.norm(q1));
q1.reshape(-1,1);
r   = q1.dot(X_b[:,0]);
Q   = q1.reshape(-1,1);
R   = np.array(r).reshape(1,1);
y_hat = Q.dot(Q.T).dot(y_train);
y_new = y_new-y_hat;
rss_man = np.sum(np.square(y_train-y_hat));
b_man = np.linalg.inv(R).dot(Q.T).dot(y_train);
selected_idx = list();
selected_idx.append(0);
best_fw_qr_models = {num_features : dict()};
best_fw_qr_subset = dict();
best_fw_qr_subset['names']          = ["intercept",];
best_fw_qr_subset['indexes']        = [0]; #wrt X_with_bias_coeffs
best_fw_qr_subset['coeffs']         = b_man.copy();
best_fw_qr_subset['err']            = rss_man;
best_fw_qr_subset['all_errors']     = [];
best_fw_qr_subset['skl_model']      = lr;
best_fw_qr_subset['indexes']        = [0,];
best_fw_qr_models[0]                = best_fw_qr_subset.copy();
best_fw_qr_models["error_sequence"] = list();
best_fw_qr_models["error_sequence"].append(best_fw_qr_subset['err']);
for num_features in np.arange(1,len(X_train.columns)+1):
    best_ = -np.infty;
    errors = {"features":list(), "errors":list()};
    best_fw_qr_subset['names'].append('');
    best_fw_qr_subset['indexes'].append(np.infty);
    for ff in l_feat:
        idx = X_train.columns.get_loc(ff)+1;
        #take the new features
        x_k = X_b[:,idx].reshape(-1,1);
        #project the new feature onto the feature sapce and take the residual
        residual = x_k - Q.dot(Q.T.dot(x_k));#x_k.T.dot(Q).dot(Q.T).T;
        #Once normalized the residual becomes a new axis
        q_new = residual/scipy.linalg.norm(residual);
        # The next feature is the one for which the projection of y onto
        # itself has the biggest value.
        y_q_prod_sq = np.square(y_new.T.dot(q_new));
        #update Q
        Q_temp = np.concatenate((Q,q_new), axis=1);
        #expand R by adding a row of 0 at the end
        R_temp = np.concatenate((R,np.zeros((1,R.shape[1]))), axis=0);
        #expand R by adding a column at the end with new projection
        R_temp = np.concatenate((R_temp, x_k.T.dot(Q_temp).T), axis =1)
        y_hat = Q_temp.dot(Q_temp.T).dot(y_train);
        rss_man = np.sum(np.square(y_train-y_hat));
        errors["features"].append(ff);
        errors['errors'].append(rss_man);
        if (y_q_prod_sq> best_):
            q_best = q_new;
            r_best = residual;
            best_ = y_q_prod_sq;
            best_ff = ff;
            best_fw_qr_subset['err'] = rss_man;
            x_best = x_k;

    Q = np.concatenate((Q,q_best), axis=1);
    R = np.concatenate((R,np.zeros((1,R.shape[1]))), axis=0);
#    R = np.concatenate((R,x_k.T.dot(Q)), axis =1)
    R = np.concatenate((R,Q.T.dot(x_best)), axis=1)
    #here is the difference from normal forward stepwise
    y_hat = Q.dot(Q.T).dot(y_train);
    y_new = y_new - y_hat;#
    b_man = np.linalg.inv(R.T.dot(R)).dot(R.T).dot(Q.T).dot(y_train);
    best_fw_qr_subset['names'][-1]   = best_ff;
    best_fw_qr_subset['indexes'][-1] = idx; #wrt X_with_bias_coeffs
    best_fw_qr_subset['coeffs']      = b_man.copy();
    best_fw_qr_subset['all_errors']  = errors;
    l_feat.remove(best_ff);
    best_fw_qr_models["error_sequence"].append(best_fw_qr_subset['err']);
    best_fw_qr_models[num_features] = best_fw_qr_subset.copy();
print(best_fw_qr_models[5]['coeffs'])

(67, 9)
[2.45234509 0.58245277 0.26675877 0.24100319 0.19918756 0.12594814]


In [41]:
best_fw_qr_models[5]['coeffs']


array([2.45234509, 0.58245277, 0.26675877, 0.24100319, 0.19918756,
       0.12594814])

In [42]:
best_models[4]['coeffs']

array([2.45234509, 0.62345   , 0.25488273, 0.20339287, 0.2800554 ])

In [46]:
class singleFWModel(best_submodel_abstract):
    def __init__(self, order : int, currFeatures=tuple(), feature=""):
        super(singleFWModel, self).__init__();
        self.currentFeatures = currFeatures;
        self.last_addedFeatures = feature;
        self.order = order;
    def getAllFeatures(self):
        return self.currentFeatures + (self.last_addedFeatures,);
   
    def _update_best_model(self, pipeline, subset, mse, col_names,):
        super(singleFWModel, self)._update_best_model(pipeline, subset, mse, col_names);
        self.last_addedFeatures = col_names[-1];

    def fit(self, X, y, isItwithoutBias=True):
        X_no_bias   = self._removeBias(X, len(self.currentFeatures), isItwithoutBias);
        if self.column_names and self.currentFeatures:
            self.currFeatureIdx  = tuple([ii for (ii,el) in enumerate(self.column_names) if el in self.currentFeatures]);
        else:
            self.currFeatureIdx = self.currentFeatures;
        errors = error_sequence();
        if not self.currentFeatures:
            temp_pipe, mse_sk = self.fit1Step(np.ones((X_no_bias.shape[0],1)), y, (0,));
            errors.addNewElement(("intercept",), mse_sk, (0,));
            self._update_best_model(temp_pipe, None, mse_sk, tuple(('intercept',)));
            return;
        err = np.infty;
        used_features_idx = list(self.currFeatureIdx);
        used_features_idx.append(-1);#append a fake element that is deleted in the for
        errors = error_sequence();
        unused_features = tuple([el for el in self.column_names if el not in self.currentFeatures]);
        for ff in unused_features:
            used_features_idx = used_features_idx[:-1]; #discard the last added
            used_features_idx.append(self.column_names.index(ff));
            # copy to have the same type instead of declaring
            temp_pipe, mse_sk = self.fit1Step(X_no_bias, y, tuple(used_features_idx));
            errors.addNewElement(self.column_names, mse_sk, used_features_idx);
            if mse_sk < err:
                err = mse_sk;
                self._update_best_model(temp_pipe, used_features_idx,
                                        mse_sk, list(self.currentFeatures) + [ff,]);
        self.all_errors = errors;

class FWComponentMethod:
    def updateTarget(self, y, model=None, X=None):
        return y;

class OrthogonalFWComponentMethod(FWComponentMethod):
    def updateTarget(self, y, model=None, X=None):
        return (y-model.predit(X));
        
#Decorator pattern
class best_fw_sk(AllBestModels, FWComponentMethod):
    def __init__(self, stepMethod : FWComponentMethod):
        super(best_fw_sk, self).__init__();
        self._componentStepMethod = stepMethod;
    
    def fit(self, X, y, num_features, isItwithoutBias=True):
        used_features = list();
        y_new = y;
        for num_features in np.arange(0,X.shape[1]+1):
            temp = singleFWModel(num_features, currFeatures=tuple(used_features));
            temp.fit(X, y_new);
            y_new = self.updateTarget(y_new, temp, X);
            self.bestModels[num_features] = temp;
            used_features =  used_features + [temp.last_addedFeatures,];


### NOTE: `singleFWModel` has some useless attributes coming from best_subset

In [49]:
stdFit = FWComponentMethod();
orthFit = OrthogonalFWComponentMethod();
l = best_fw_sk(stdFit);
l.fit(X_train, y_train, 5);
print(l.bestModels[4].model.get_intercept_coef());
#array([2.45234509, 0.64128976, 0.34852683, 0.22422118])
#array([2.45234509, 0.62345   , 0.25488273, 0.20339287, 0.2800554 ]) 
#array([2.45234509, 0.58245277, 0.26675877, 0.19918756, 0.24100319, 0.12594814]) #best model[5]

orthFit = OrthogonalFWComponentMethod();
l2 = best_fw_sk(stdFit);
l2.fit(X_train, y_train, 5);
print(l2.bestModels[4].model.get_intercept_coef());

[2.45234509 0.62345    0.25488273 0.2800554  0.20339287]
[2.45234509 0.62345    0.25488273 0.2800554  0.20339287]


## Orthogonal dependent fw stepwise class

In [ ]:
class
#here is the difference from normal forward stepwise
    y_new = y_new - y_pred;
  #  l_feat.remove(best_fw_orth_dep_subset['names'][-1]);
   # best_fw_orth_dep_models["error_sequence"].append(best_fw_orth_dep_subset['err']);
    #best_fw_orth_dep_models[num_features] = best_fw_orth_dep_subset.copy();


# Backword stepwise

In [69]:
#sklearn does not support
import itertools;
from scipy.linalg import inv
num_features = X_b.shape[1]-1;
#intercept only
X_small = X_b.copy();#np.copy(X_b[:,0]).reshape(-1,1); #np.ones
b_man  = scipy.linalg.inv(X_small.T.dot(X_small)).dot(X_small.T.dot(y_train));
lr = LinearRegression();
lr.fit(X_small, y_train);
y_pred_sk  = lr.predict(X_small);
print(X_small.shape, b_man.shape)
y_pred_man = X_small.dot(b_man.reshape(-1,1));
rss_sk  = np.sum(np.square(y_train-y_pred_sk));
rss_man = np.sum(np.square(y_train-y_pred_man.reshape(y_train.shape)));
best_bw_models = {num_features : dict()};
best_bw_subset = dict();
best_bw_subset['names']      = list(X_train.columns);
best_bw_subset['indexes']    = [ii for ii in np.arange(0,num_features+1)]; #wrt X_with_bias_coeffs
best_bw_subset['coeffs']     = b_man;
best_bw_subset['err']        = rss_man;
best_bw_subset['all_errors'] = [];
best_bw_subset['skl_model']  = lr;
best_bw_models[num_features] = best_bw_subset.copy();
best_bw_models["error_sequence"] = list();
best_bw_models["z_scores"] = list();
best_bw_models["error_sequence"].append(best_fw_subset['err']);
#best_bw_subset['indexes'] = [np.arange(0,num_features+1)];
orig_num_features = num_features;
indexes = [ii for ii in range(0,10)];
for num_features in np.arange(1,len(X_train.columns)+1):
    XX = scipy.linalg.inv(X_small.T.dot(X_small));
    z_score_worst = np.infty;
    errors = {"features":list(), "errors":list()};
    normalized_rss = rss_man/(len(X_train-orig_num_features+num_features -1))
    
    for k,ff in enumerate(best_bw_subset['names']):
        z_score = b_man[k]/(np.sqrt(normalized_rss*XX[k,k]));
        best_bw_models["z_scores"].append(z_scores);
        if abs(z_score) < z_score_worst:
            z_score_worst = z_score;
            k_worst = k;
            ff_worst = ff;
            best_bw_subset['err'] = rss_man;

    best_bw_subset['names'].remove(ff_worst);
    best_bw_subset['indexes'].remove(X_train.columns.get_loc(ff_worst)+1);#wrt X_with_bias_coeffs
    y_pred_man = X_small.dot(b_man);   
    rss_man = np.sum(np.square(y_train-y_pred_man));
    X_small = X_b[:, best_bw_subset['indexes']];
    b_man  = scipy.linalg.inv(X_small.T.dot(X_small)).dot(X_small.T.dot(y_train));
    best_bw_subset['coeffs'] = b_man.copy();
    best_bw_subset['err']    = rss_man;
    best_bw_models[orig_num_features-num_features] = best_bw_subset.copy();
    best_bw_models["error_sequence"].insert(0, best_bw_subset["err"]);
    


#Now it contains only 0

X_small = X_b[:, best_bw_subset['indexes']];
b_man  = scipy.linalg.inv(X_small.T.dot(X_small)).dot(X_small.T.dot(y_train));
y_pred_man = X_small.dot(b_man);   
rss_man = np.sum(np.square(y_train-y_pred_man));
print(rss_man)
best_bw_subset['err']    = rss_man;
best_bw_models["error_sequence"].insert(0, best_bw_subset["err"]);
XX = scipy.linalg.inv(X_small.T.dot(X_small));
errors = {"features":list(), "errors":list()};
normalized_rss = rss_man/(len(X_train-orig_num_features+num_features -1))
z_score = b_man[k]/(np.sqrt(normalized_rss*XX[k,k]));
best_bw_subset['names'] = ['intercept',]
        

(97, 9) (9,)
127.91765888525164


In [70]:
best_bw_models[4]["coeffs"]

array([ 2.47838688,  0.70243067,  0.32512188, -0.12620984,  0.18030652])

In [71]:
best_fw_models[4]["coeffs"]

array([2.47838688, 0.61967302, 0.22845699, 0.29055615, 0.11344257])

# Stagewise regression

In [72]:
#sklearn does not support
import itertools;
from scipy.linalg import inv


print(X_b.shape)
#intercept only
X_small = np.copy(X_b[:,0]).reshape(-1,1); #np.ones
b_man  = scipy.linalg.inv(X_small.T.dot(X_small)).dot(X_small.T.dot(y_train));

y_pred_man = X_small.dot(b_man);
rss_man = np.sum(np.square(y_train-y_pred_man.reshape(y_train.shape)));
best_sw_models = dict();

residual = y_train - y_pred_man;
best_sw_models['coeffs'] = np.concatenate((b_man.reshape(1,1), np.zeros((X_train.shape[1],1))), axis=0);
#best_sw_models['error_sequence'] = list();
#best_sw_models['error_sequence'].append(residual);
#best_sw_models['coeffs_history'] = list();
#best_sw_models['coeffs_history'].append(b_man);

#best_sw_models['coeffs_updated'] = list();
#best_sw_models['coeffs_updated'].append(0);
count = 0;

l_feat = list(X_train.columns);
while 1:
    #print(residual)
    max_corr = 0;
    for ff in l_feat:
        idx = X_train.columns.get_loc(ff)+1;
        x_k = X_b[:,idx];
        corr = x_k.dot(residual)/len(x_k);
        if abs(corr) > abs(max_corr):
           max_idx = idx;
           max_corr = corr;
    print(max_corr)
    if abs(max_corr)<0.001 or count>10000:
        break;
    print(max_corr)
    x_most_corr = X_b[:, max_idx];
    b_most_corr = (x_most_corr.T.dot(x_most_corr))**(-1)*(x_most_corr.T.dot(residual));
    best_sw_models['coeffs'][max_idx] = best_sw_models['coeffs'][max_idx] + b_most_corr;
    y_hat = X_b.dot(best_sw_models['coeffs']);
    residual = y_train.values.reshape(-1,1) - y_hat;
    #best_sw_models['error_sequence'].append(residual);
    count += 1;

(97, 9)
0.8434274382607599
0.8434274382607599
[0.26100874]
[0.26100874]
[0.15519087]
[0.15519087]
[-0.15684236]
[-0.15684236]
[0.08818995]
[0.08818995]
[-0.09316011]
[-0.09316011]
[0.11129984]
[0.11129984]
[0.06466917]
[0.06466917]
[-0.06737883]
[-0.06737883]
[0.04535345]
[0.04535345]
[-0.04331858]
[-0.04331858]
[-0.02499753]
[-0.02499753]
[0.03693757]
[0.03693757]
[-0.02350057]
[-0.02350057]
[0.02171228]
[0.02171228]
[0.01954552]
[0.01954552]
[-0.02046127]
[-0.02046127]
[0.02127011]
[0.02127011]
[-0.01473005]
[-0.01473005]
[-0.00906994]
[-0.00906994]
[0.01479569]
[0.01479569]
[-0.00995915]
[-0.00995915]
[0.00901681]
[0.00901681]
[0.00547287]
[0.00547287]
[-0.00460382]
[-0.00460382]
[0.00738321]
[0.00738321]
[-0.00496972]
[-0.00496972]
[0.00334517]
[0.00334517]
[-0.00369254]
[-0.00369254]
[0.0031471]
[0.0031471]
[-0.00229807]
[-0.00229807]
[0.00306234]
[0.00306234]
[-0.00162514]
[-0.00162514]
[-0.0013691]
[-0.0013691]
[0.00162386]
[0.00162386]
[0.0011997]
[0.0011997]
[-0.00111989]
[-0.

In [73]:
best_sw_models['coeffs']

array([[ 2.47838688],
       [ 0.66196462],
       [ 0.26415584],
       [-0.15652642],
       [ 0.13968469],
       [ 0.31190695],
       [-0.14510743],
       [ 0.0343617 ],
       [ 0.12512751]])

In [74]:
best_models[8]['coeffs']

array([ 2.47838688,  0.6617092 ,  0.26510309, -0.15737767,  0.13958604,
        0.31369926, -0.14751935,  0.03536545,  0.1250701 ])